In [1]:
import pathlib
import subprocess
from tqdm import tqdm
import pandas as pd
import ray
import re

In [3]:
subclass_annot = pd.read_pickle('/browser/metadata/CEMBA_snmC.cell_metadata.pickle')
cell_subclass_set = set(subclass_annot['CellSubClass'].unique())

In [4]:
cmd = 'sudo docker exec higlass-container python higlass-server/manage.py list_tilesets'

cur_tilesets = subprocess.run(
            cmd,
            shell=True,
            check=True,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            encoding="utf8",
        ).stdout

In [5]:
p = re.compile('tileset: Tileset \[name: (?P<name>.+)] \[ft: (?P<file_type>.+)] \[uuid: (?P<uuid>.+)]')
records = {}
for record in cur_tilesets.split('\n'):
    match = p.search(record)
    if match is None:
        print(record)
        continue
    name = match.group('name')
    file_type = match.group('file_type')
    uuid = match.group('uuid')
    records[name] = {'file_type': file_type, 'uuid': uuid}
curr_tliesets_df = pd.DataFrame(records).T

In [14]:
name_to_ct = {}
name_to_type = {}
for n in curr_tliesets_df.index:
    _l = n.split(' ')
    ct = ' '.join(_l[:-2])
    track_type = ' '.join(_l[-2:])
    if ct not in cell_subclass_set:
        print(ct)
        ct = ''
    name_to_ct[n] = ct
    name_to_type[n] = track_type
curr_tliesets_df['CellSubClass'] = name_to_ct
curr_tliesets_df['TrackType'] = name_to_type

mm10 main
CEMBA snm3C Domain


In [17]:
curr_tliesets_df['TrackType'].value_counts()

ATAC CPM             274
mCH Frac             274
mCG Frac             274
Domain Boundary      273
Compartment Score    273
Impute 100K          273
Impute 10K           273
Raw 100K             273
chrom sizes            1
Chi2 Stats             1
Name: TrackType, dtype: int64

In [19]:
curr_tliesets_df.to_csv('/browser/metadata/HiglassTracks.csv.gz')

In [20]:
curr_tliesets_df

,file_type,uuid,CellSubClass,TrackType
mm10 main chrom sizes,chromsizes-tsv,ZQZRPdYDSFmTeTeXDYSmbA,,chrom sizes
CA3 Glut ATAC CPM,bigwig,bt2BX-WeRneyzeT-xZOF5g,CA3 Glut,ATAC CPM
L6 CT CTX Glut ATAC CPM,bigwig,cAqqyvTTTKW2hOCR0MGLFA,L6 CT CTX Glut,ATAC CPM
Sst Gaba ATAC CPM,bigwig,fA7Rwv3MRn6g9gZpjZAhNg,Sst Gaba,ATAC CPM
STR D1 Gaba ATAC CPM,bigwig,FUvc9xtdSJixiIeHHY5diQ,STR D1 Gaba,ATAC CPM
...,...,...,...,...
PMv-TMv Pitx2 Glut Raw 100K,cooler,dWAGWXxURVK4fEpzA_y5MQ,PMv-TMv Pitx2 Glut,Raw 100K
MM-ant Foxb1 Glut Raw 100K,cooler,eHyQMVuCQCibJYByOsrMnQ,MM-ant Foxb1 Glut,Raw 100K
IPN-LDT Vsx2 Nkx6-1 Glut Raw 100K,cooler,A_Hh9emeSiGIiUX3-R1N1A,IPN-LDT Vsx2 Nkx6-1 Glut,Raw 100K
Peri NN Raw 100K,cooler,HCTZBfLDRfK8VaAsWrF92w,Peri NN,Raw 100K
